In [1]:
import os, json
import pandas as pd
import timeit
import sqlalchemy as alch
import re
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.geometry import shape
import fiona
import pyproj

In [2]:
%%time
# OPEN THE JSON FILES AND STORE THE DATA


path = '.'
#json_files = [os.path.join(root, name)
#       for root, dirs, files in os.walk(path)
#       for name in files
#       if name.endswith(".json")]
json_files = ['.\\loc_file_all.json']


Wall time: 0 ns


In [3]:
print(json_files)

['.\\loc_file_all.json']


In [4]:
%%time

entries = []

#open all files, count entries and add relevant info to the list
for file in json_files:
    with open(file, 'rb') as f:
        data = f.readlines()
        data = [json.loads(line) for line in data]
        entries = entries + data

Wall time: 25.5 s


In [5]:
count = len(entries)
print(count)

3114612


In [6]:
%%time
keys = [None]*count
values = [None]*count
i = 0
for entry in entries:
    for key in entry.keys():
        keys[i]= key
        
    for value in entry.values():
        typ = value[0][2][0][4]
        coords = value[0][2][0][1]
        #obj is type and coord
    obj = [typ, coords]
    values[i] = obj
    i+=1

Wall time: 4.15 s


In [7]:
%%time

locs = [None]*count

#cds_system = pyproj.CRS("EPSG:27700")
#wgs84=pyproj.CRS("EPSG:4326")

#transformer = pyproj.Transformer.from_crs(wgs84, "epsg:27700")


i = 0
bx = 0
for value in values:
    # get jobid
    jobid = keys[i]
    
    # get coordinates
    if value[0] == 'box':
        bx += 1 
        coords =  re.findall('\(.*?\)',value[1])
        
        latsum = 0
        lngsum = 0
        c = 0
        for coord in coords:
            lat, lng = map(float, coord.strip('()').split(','))
            latsum += lat
            lngsum += lng
            c+=1
        
        x = latsum/c
        y = lngsum/c
        
        # to place all boxes out of the uk for a test
        #x = 0
        #y = 0
        
    else:
        x, y = map(float, coord.strip('()').split(',')) 
    
    
    #x, y = transformer.transform(x, y)
    
    cds = [x,y]
    locs[i] = [jobid, cds]
    i+=1


Wall time: 33.4 s


In [8]:
print(locs[:10])

[['37000000', [52.3234292, -0.1927723]], ['37000001', [51.231523679999995, -0.48607088000000004]], ['37000002', [53.38394864000001, -1.6107502999999999]], ['37000003', [51.4753219, -0.15964740000000002]], ['37000004', [53.47067634, -2.40216138]], ['37000005', [51.331284499999995, -0.7540981]], ['37000006', [51.4753219, -0.15964740000000002]], ['37000007', [52.40423952, -1.5382583]], ['37000008', [51.45614295999999, -2.6351899199999997]], ['37000009', [51.4753219, -0.15964740000000002]]]


In [9]:
%%time
with open('NUTS_Level_3__January_2018__Boundaries.GEOJSON', 'r') as f:
        geo_data = f.read()
        geo_data = json.loads(geo_data)

        

Wall time: 6.64 s


In [10]:
%%time

nutscodes = []
nutsnames = []
polygons = []


for f in geo_data['features']:

    nutscodes.append(f['properties']['nuts318cd'])
    nutsnames.append(f['properties']['nuts318nm'])
    poly = shape(f['geometry'])
    polygons.append(poly)
    
    #poly = shape(collection['geometry'])
    #polygons.append(poly)
    

Wall time: 256 ms


In [11]:
areas = [None]*179
for i in range(179):
    areas[i] = [nutsnames[i], nutscodes[i], polygons[i]]

In [12]:
print(areas)

[['Hartlepool and Stockton-on-Tees', 'UKC11', <shapely.geometry.multipolygon.MultiPolygon object at 0x0000026AA4FC1BE0>], ['South Teesside', 'UKC12', <shapely.geometry.multipolygon.MultiPolygon object at 0x0000026AD0A45048>], ['Darlington', 'UKC13', <shapely.geometry.polygon.Polygon object at 0x0000026AD0A451D0>], ['Durham CC', 'UKC14', <shapely.geometry.polygon.Polygon object at 0x0000026AD0A452E8>], ['Northumberland', 'UKC21', <shapely.geometry.multipolygon.MultiPolygon object at 0x0000026AD0A45208>], ['Tyneside', 'UKC22', <shapely.geometry.multipolygon.MultiPolygon object at 0x0000026AD0A453C8>], ['Sunderland', 'UKC23', <shapely.geometry.multipolygon.MultiPolygon object at 0x0000026AD0A454E0>], ['West Cumbria', 'UKD11', <shapely.geometry.multipolygon.MultiPolygon object at 0x0000026AD0A45470>], ['East Cumbria', 'UKD12', <shapely.geometry.multipolygon.MultiPolygon object at 0x0000026AD0A45550>], ['Manchester', 'UKD33', <shapely.geometry.polygon.Polygon object at 0x0000026AD0A45668>],

In [13]:
job_loc_pairs = [None]*len(locs)

In [21]:
%%time
# count the number oof entries of each type
types = {}
for value in values:
    try:
        types[value[0]] = types[value[0]]+1
        
    except:
        types[value[0]] = 1
        print("added "+ value[0]+" type")
print(types)

added box type
added country type
added pt type
{'box': 3113730, 'country': 848, 'pt': 34}
Wall time: 591 ms


In [14]:
%%time
i=0

for loc in locs:
    pt = Point(loc[1][::-1])
    code = 'NA'
    name = 'NA'
    for area in areas:
        if area[2].contains(pt):
            code = area[1]
            name = area[0]
    job_loc_pairs[i] = {'jobid':loc[0], 'nutsCode':code, 'nutsName': name}
    i+=1
    if i %100000 == 0:
            print(i,"entries parsed")

100000 entries parsed
200000 entries parsed
300000 entries parsed
400000 entries parsed
500000 entries parsed


KeyboardInterrupt: 

In [18]:
# connect to the database
engine_new = alch.create_engine('sqlite:///jobs_nutscodes_names_v3.db', echo=False)

In [19]:
metadata = alch.MetaData()
jobs = alch.Table('jobs', metadata,
              alch.Column('jobid', alch.Integer, primary_key=True),
              alch.Column('nutsCode', alch.String(5)),
              alch.Column('nutsName', alch.String(64))
             )
metadata.create_all(engine_new)

In [20]:
conn = engine_new.connect()

In [21]:
%%time
conn.execute(jobs.insert(), job_loc_pairs)

Wall time: 1min 30s
